In [12]:
import datasets
train_dataset = datasets.load_dataset('parquet',data_files="https://huggingface.co/datasets/iohadrubin/nq/resolve/main/data/train-00000-of-00012-aebee16ac9d5ed6f.parquet",split="train")

In [30]:
%env JAVA_HOME /usr/lib/jvm/java-8-openjdk-amd64/


env: JAVA_HOME=/usr/lib/jvm/java-8-openjdk-amd64/


In [31]:
# from datasets.parallel import parallel_backend
# from datasets import load_dataset
# with parallel_backend('spark'):
#       dataset = load_dataset("iohadrubin/nq", num_proc=12)

/home/ohadr/.local/lib/python3.8/site-packages/datasets/utils/experimental.py:36: UserWarning: 'parallel_backend' is experimental and might be subject to breaking changes in the future.
  warnings.warn(


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/04 02:22:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting num_proc from 12 to 2 for the validation split as it only contains 2 shards.


Generating validation split:   0%|          | 0/6515 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/58880 [00:00<?, ? examples/s]

In [20]:

def unstack_element(element,n_examples=None):
    keys = list(element.keys())
    if n_examples is None:
        n_examples = len(element[keys[0]])
    for i in range(n_examples):
        micro_element = {}
        for key in keys:
            try:
                micro_element[key] = element[key][i]
            except:
                print([(key,len(element[key])) for key in keys])
                raise
        yield micro_element

In [22]:
def tokenize_train(example,query_field="query",pos_field="positive_passages",neg_field="negative_passages"):
    tokenize = lambda x: {"input_ids":x}
    query = example[query_field]
    pos_psgs = [p['title'] + " " + p['text'] for p in unstack_element(example[pos_field])]
    neg_psgs = [p['title'] + " " + p['text'] for p in unstack_element(example[neg_field])]

    example['query_input_ids'] = dict(tokenize(query))
    example['pos_psgs_input_ids'] = [dict(tokenize(x)) for x in pos_psgs]
    example['neg_psgs_input_ids'] = [dict(tokenize(x)) for x in neg_psgs]

    return example
from functools import partial
train_data = train_dataset.map(
    partial(tokenize_train,query_field="question",pos_field="positive_ctxs",neg_field="hard_negative_ctxs"),
    batched=False,
    # num_proc=20,
    desc="Running tokenizer on train dataset",
)

Running tokenizer on train dataset:   0%|          | 0/4907 [00:00<?, ? examples/s]

In [25]:
train_data["query_input_ids"][1]

{'input_ids': 'who sang waiting for a girl like you'}

In [ ]:
# dataset.flatten()[0]

In [10]:
for x in dataset.flatten()[0].keys():
    print(x)

dataset
question
answers
positive_ctxs.title
positive_ctxs.text
positive_ctxs.score
positive_ctxs.title_score
positive_ctxs.passage_id
negative_ctxs.title
negative_ctxs.text
negative_ctxs.score
negative_ctxs.title_score
negative_ctxs.passage_id
hard_negative_ctxs.title
hard_negative_ctxs.text
hard_negative_ctxs.score
hard_negative_ctxs.title_score
hard_negative_ctxs.passage_id


In [11]:
dataset.flatten()[0]

{'dataset': 'nq_train_psgs_w100',
 'question': 'big little lies season 2 how many episodes',
 'answers': ['seven'],
 'positive_ctxs.title': ['Big Little Lies (TV series)',
  'Little People, Big World',
  'Big Little Lies (TV series)',
  'Big Little Lies (TV series)',
  'Andrea Arnold',
  'Designing Women',
  'Big Little Lies (TV series)',
  'The X-Files (season 2)',
  'The Big Bang Theory'],
 'positive_ctxs.text': ['series garnered several accolades. It received 16 Emmy Award nominations and won eight, including Outstanding Limited Series and acting awards for Kidman, Skarsgård, and Dern. The trio also won Golden Globe Awards in addition to a Golden Globe Award for Best Miniseries or Television Film win for the series. Kidman and Skarsgård also received Screen Actors Guild Awards for their performances. Despite originally being billed as a miniseries, HBO renewed the series for a second season. Production on the second season began in March 2018 and is set to premiere in 2019. All seve